## Machine-Learning
---
We will be using three types of models to predict prices of listing, and also using three types of optimization methods to improve the prediction accuracy.

### Table of Contents
1. [Modelling](#model)
2. [Optimization](#optimize)
3. [Summary](#summary)

In [1]:
random_state = 9999
image_output_params = {'width': 1080, 'height': 600, 'scale': 6}
render = 'svg' #or None to have interactive plots

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.inspection import permutation_importance


import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/clean_listingfinal.csv', index_col=0)
# Split data and drop unnecessary data
Y = df['price']
X = df.drop(['price'], axis=1)
X = X.astype('float64')
X.head()
# Split data
XXyy = X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=random_state)
# Storage
result = {'Method':[], 'Train R2':[], 'Train RMSE':[], 'Test R2': [], 'Test RMSE': []}

In [4]:
f"The standard deviation of price of listing is ${df['price'].std():.2f}."

'The standard deviation of price of listing is $132.36.'

In [5]:
# """ Helper functions """
def update_frame(frame: dict, y_train, y_train_pred, y_true, y_pred, method_name):
    frame['Method'].append(method_name)
    frame['Train R2'].append(r2_score(y_train, y_train_pred))
    frame['Train RMSE'].append(np.sqrt(mean_squared_error(y_train, y_train_pred)))
    frame['Test R2'].append(r2_score(y_true, y_pred))
    frame['Test RMSE'].append(np.sqrt(mean_squared_error(y_true, y_pred)))

def plot(y_true, y_pred, title: str, band=True):
    """Plot predicted value against actual value, with formatting added to figure

    Keyword arguments:
    y_true: list like, x axis
    y_pred: list like, y axis
    title: string, added to top-center of figure
    band: boolean, If true, add two band lines to the figure with RMSE distance from y=x line
    """
    h = pd.DataFrame(data={'Actual y':y_true, 'Predicted y': y_pred})
    
    fig = go.Figure()
    fig = px.scatter(h, x='Actual y', y='Predicted y', title=title)
    fig.add_trace(go.Scatter(x=[0, 600],y=[0, 600],mode="lines",line=go.scatter.Line(color='gray'),showlegend=False)) #Diagonal Line
    if band:
        RMSE = df['price'].std()
        fig.add_traces([
            go.Scatter(x=[0, 600],y=[0+RMSE, 600+RMSE],mode='lines',line=go.scatter.Line(dash='dot'),showlegend=False),
            go.Scatter(x=[0, 600],y=[0-RMSE, 600-RMSE],mode='lines',line=go.scatter.Line(dash='dot'),showlegend=False)
        ])
    fig.update_layout(width=700,height=500)
        
    pio.show(fig)
    return fig

def simulate(model, XXyy, name: str):
    """Simulate performance metrics of model using R2 and RMSE and update global result and save plot image

    Keyword arguments:
    model: Implements Skleanr API and must be fitted
    XXyy: output of train_test_split
    """
    # Model must be fitted and implement predict, score methods.
    # XXyy: X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = XXyy
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    update_frame(result, y_train, y_train_pred, y_test, y_pred, name)

    print("-"*30)
    print(f"Train Score: {r2_score(y_train, y_train_pred): .3f}")
    print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)): .3f}")

    print("-"*30)
    print(f"Test Score: {r2_score(y_test, y_pred): .3f}")
    print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)): .3f}")
    
    fig = plot(y_test, y_pred, name)
    # fig.write_image(f'images/{name}.png', **image_output_params)

def feature_plot(feature_name, feature_score, name: str):
    feature_importance = pd.DataFrame({'Feature': feature_name, 'Score': feature_score})
    feature_importance.sort_values(by='Score', axis=0, inplace=True, ascending=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=feature_importance.head(15)['Score'], y=feature_importance.head(15)['Feature'], orientation='h'))
    fig.update_layout(title={'text': f'{name} Most Relevant Feature'}, xaxis_title='Score', yaxis_title='Feature', height=500, width=700)
    pio.show(fig)
    # fig.write_image(f'images/{name} Most Relevant Feature.png', **image_output_params)

    fig = go.Figure()
    fig.add_trace(go.Bar(x=feature_importance.tail(15)['Score'], y=feature_importance.tail(15)['Feature'], orientation='h'))
    fig.update_layout(title={'text': f'{name} Least Relevant Feature'}, xaxis_title='Score', yaxis_title='Feature', height=500, width=700)
    pio.show(fig)
    # fig.write_image(f'images/{name} Least Relevant Feature.png', **image_output_params)

---
<a id='model'></a>
## 1.  Modelling
To find a model that best predicts price. We will being trying :
- linear model: [Linear Regression](#model-1-linear-regression)
- tree-based model: [HistGradientBoostRegressor](#model-2-gradient-boost-regressor), [XGBoost](#model-3-xgboost-regression)
- and to try something new: [Neural Network](#model-4-neural-networks).

We will be using $R^2$ score to compare the goodness of the model, and RMSE to find the accuracy of the prediction.

### Model 1: Linear Regression
We first use linear regression, a method taught for predicting numerical values.

In [6]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

simulate(model, XXyy, 'Linear Regression')
feature_plot([col for col in X.columns], [model.coef_[i].round(5) for i in range(len(X.columns))], 'Linear Regression')

------------------------------
Train Score:  0.638
Train RMSE:  80.135
------------------------------
Test Score:  0.641
Test RMSE:  76.829


### Model 2: Gradient Boost Regressor
We can use gradient boosting - a type of ensemble machine learning algorithms. Ensemble is a collection of decision trees. Every iteration, a decision tree is added to model to minimise the error. Models are fit using any arbitrary differentiable loss function and gradient descent optimization algorithm, and the goal is to minimise loss gradient. We will be using HistGradientBoostingRegressor from sklearn.

In [7]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(random_state=random_state, max_iter=1000)

model.fit(X_train, y_train)

simulate(model, XXyy, name='Gradient Boost Regressor')

feature_plot(model.feature_names_in_, 
             permutation_importance(model, X_test, y_test, random_state=random_state).importances_mean, 
             'Gradient Boost Regressor')

------------------------------
Train Score:  1.000
Train RMSE:  1.387
------------------------------
Test Score:  0.860
Test RMSE:  48.055


### Model 3: XGBoost Regression
Similarly to gradient boosting, we can use XGBoost Regressor to model price. We will use XGBoost API to do the model fitting.

In [8]:
import xgboost as xgb

model = xgb.sklearn.XGBRegressor(tree_method='hist', objective='reg:squarederror', n_estimators = 2500,
                         learning_rate=0.01, random_state=random_state)

model.fit(X_train, y_train)

simulate(model, XXyy, 'XGBoost Regressor')

booster = model.get_booster()
feature_plot(booster.get_score().keys(), booster.get_score().values(), 'XGBoost Regressor')

------------------------------
Train Score:  0.995
Train RMSE:  9.809
------------------------------
Test Score:  0.862
Test RMSE:  47.632


### Model 4: Neural Networks

We will be using Sklearn MLPRegressor, short for Multi-layer Perception Regressor. Model optimizes for lowest squarer error by gradient descent.

In [9]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(random_state=random_state, max_iter=1000).fit(X_train, y_train)

simulate(model, XXyy, 'MLP Regressor')

------------------------------
Train Score:  0.863
Train RMSE:  49.355
------------------------------
Test Score:  0.822
Test RMSE:  54.114


We can conclude that HistGradientBoostingRegressor and XGBoost Regressor achieve the best results, with the lowest RMSE (~50) and the higher R^2 score (~0.85). All other models achieve decent results, with RMSE around half of standard deviation, and R^2 > 0.5.

---
<a id='optimize'></a>
## 2. Optimising models

We will be using RandomizedSearchCV, GridSearchCV, and Hyperopt to find the optimal parameters.
- RandomizedSearchCV is not exhuastive but covers a wide range of hyperparameters.
- GridSearchCV is exhuastive but only convers the range of hyperparameters provided.
- Hyperopt uses TPE algorithm to try hyperparameters values more likely to obtain a better result.

In [10]:
from scipy.stats import loguniform
# Code from scikit-learn-mooc
class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

### Tuning GradientBoostingRegressor with RandomizedSearchCV
Parameters to tune are obtained by consulting the documentation.

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

model = HistGradientBoostingRegressor(random_state=random_state)

param_distributions = {
    'max_bins': loguniform_int(2, 255),
    'max_leaf_nodes': loguniform_int(2, 256),
    'min_samples_leaf': loguniform_int(1, 100),
    'learning_rate': loguniform(0.001, 10),
}

cv = RandomizedSearchCV(model, param_distributions=param_distributions, n_iter=350, cv=3, n_jobs=-1, scoring=['neg_mean_squared_error', 'r2'], refit='r2') .fit(X_train, y_train)

v = pd.DataFrame(cv.cv_results_)
v.sort_values(by='rank_test_r2', inplace=True)
display(v.head(6))

print(f"Best parameters: {cv.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_bins,param_max_leaf_nodes,param_min_samples_leaf,params,split0_test_neg_mean_squared_error,...,split2_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2
100,0.352107,0.027456,0.007375,0.000251,0.049172,14,83,8,"{'learning_rate': 0.04917213242114807, 'max_bi...",-2436.841310,...,-2340.291731,-2478.501646,133.155137,1,0.859348,0.862527,0.856313,0.859396,0.002537,1
347,0.540875,0.024095,0.008521,0.002109,0.047694,85,77,1,"{'learning_rate': 0.04769438185741469, 'max_bi...",-2207.882920,...,-2602.972788,-2527.954896,236.731393,2,0.872564,0.856599,0.840186,0.856449,0.013219,2
119,0.374906,0.007654,0.009634,0.001673,0.053208,170,115,21,"{'learning_rate': 0.05320778115006037, 'max_bi...",-2273.267348,...,-2481.304138,-2556.604290,267.438778,3,0.868790,0.849244,0.847656,0.855230,0.009610,3
70,0.931132,0.027171,0.010067,0.001357,0.050859,60,148,1,"{'learning_rate': 0.05085924268513077, 'max_bi...",-2309.456166,...,-2542.643629,-2557.888760,209.345894,4,0.866701,0.854088,0.843890,0.854893,0.009330,4
45,0.141192,0.011688,0.004940,0.001367,0.073478,5,25,9,"{'learning_rate': 0.07347752782115871, 'max_bi...",-2365.973653,...,-2450.867096,-2587.948996,256.246368,6,0.863439,0.847601,0.849525,0.853521,0.007056,5
195,0.270793,0.007982,0.005503,0.000516,0.050607,84,36,5,"{'learning_rate': 0.050607387224914854, 'max_b...",-2313.813454,...,-2525.621449,-2585.992744,250.543509,5,0.866449,0.849073,0.844935,0.853486,0.009321,6


Best parameters: {'learning_rate': 0.04917213242114807, 'max_bins': 14, 'max_leaf_nodes': 83, 'min_samples_leaf': 8}


In [12]:
model = HistGradientBoostingRegressor(**cv.best_params_, random_state=random_state).fit(X_train, y_train)
simulate(model, XXyy, 'RandomSearch CV Gradient Boost Regressor')

------------------------------
Train Score:  0.978
Train RMSE:  19.716
------------------------------
Test Score:  0.873
Test RMSE:  45.802


### Tuning XGBoost Regressor

Parameters to tune:
- n estimators & learning_rate: Takes longer to achieve same error reduction, however smaller steps taken means we can find the optimal minimum
- max_leaves (default 0): By restricting max leaves, we can reduce overfitting.
- colsample_bytree (default 1): Fraction of columns to be randomly sampled, might reduce overfitting.
- subsample (default 1): Fraction of observations to sample for each tree, lower values reduce overfitting.

In [13]:
model = xgb.XGBRegressor(tree_method='hist', objective='reg:squarederror', seed=random_state)

params_grid = {
    'n_estimators': [1000, 1500],
    'learning_rate': [0.03, 0.05],
    'max_leaves': [0, 20, 40, 60, 80],
    'colsample_bytree': [0.4, 0.6, 0.8],
    'subsample': [0.6, 0.8, 1],
}

cv = GridSearchCV(estimator=model, param_grid=params_grid, cv=3, n_jobs=-1, scoring=['neg_mean_squared_error', 'r2'], refit='r2').fit(X_train, y_train)

v = pd.DataFrame(cv.cv_results_)
v.sort_values(by='rank_test_r2', inplace=True)
display(v.head(6))

print(f"Best parameters: {cv.best_params_}")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_leaves,param_n_estimators,param_subsample,params,...,split2_test_neg_mean_squared_error,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2
134,1.297294,0.037205,0.019206,0.001984,0.8,0.03,40,1000,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2449.533593,-2481.789195,113.372977,1,0.863674,0.863791,0.849606,0.859024,0.006659,1
140,1.342261,0.042738,0.019634,0.004928,0.8,0.03,60,1000,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2460.257093,-2483.571114,153.724068,2,0.866783,0.861284,0.848948,0.859005,0.007457,2
143,2.063260,0.070322,0.029069,0.007254,0.8,0.03,60,1500,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2468.393430,-2485.147595,151.979408,3,0.866787,0.861456,0.848448,0.858897,0.007702,3
137,1.982266,0.084549,0.029639,0.002633,0.8,0.03,40,1500,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2456.201627,-2485.994675,111.394310,4,0.863384,0.863743,0.849197,0.858775,0.006774,4
170,1.389383,0.025475,0.020292,0.003070,0.8,0.05,60,1000,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2425.321882,-2490.797024,144.769144,5,0.864041,0.860812,0.851093,0.858649,0.005503,5
173,2.171950,0.055197,0.026473,0.005428,0.8,0.05,60,1500,1,"{'colsample_bytree': 0.8, 'learning_rate': 0.0...",...,-2428.101231,-2491.929934,144.959578,8,0.864065,0.860759,0.850922,0.858582,0.005582,6


Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_leaves': 40, 'n_estimators': 1000, 'subsample': 1}


In [14]:
model = xgb.XGBRegressor(**cv.best_params_,random_state=random_state).fit(X_train, y_train)
simulate(model, XXyy, 'GridSearch CV XGBoost Regressor')

------------------------------
Train Score:  0.995
Train RMSE:  9.827
------------------------------
Test Score:  0.867
Test RMSE:  46.738


### Tuning both GradientBoostingRegressor and XGBoost Regressor with Hyperopt
Using Hyperopt-Sklearn, which is a wrapper of Hyperopt, a library for Distributed Asynchronous Hyper-parameter Optimization.

In [15]:
from hpsklearn import HyperoptEstimator, hist_gradient_boosting_regressor, xgboost_regression
from hyperopt import tpe

model = HyperoptEstimator(
    regressor=hist_gradient_boosting_regressor('HGBR'),
    preprocessing=[],
    algo=tpe.suggest,
    max_evals=20,
    n_jobs=-1,
    seed=random_state)
model.fit(X_train, y_train)

simulate(model, XXyy, 'Hyperopt Gradient Boost Regressor')

100%|██████████| 20/20 [00:00<00:00,  1.02trial/s, best loss: 0.18260647470669622]
------------------------------
Train Score:  0.869
Train RMSE:  48.209
------------------------------
Test Score:  0.848
Test RMSE:  50.002


In [16]:
model = HyperoptEstimator(
    regressor=xgboost_regression('XGBR'),
    preprocessing=[],
    algo=tpe.suggest,
    max_evals=10,
    n_jobs=-1,
    seed=random_state)
model.fit(X_train, y_train)

simulate(model, XXyy, 'Hyperopt XGBoost Regressor')

100%|██████████| 10/10 [00:05<00:00,  5.69s/trial, best loss: 0.14842136937025374]
------------------------------
Train Score:  0.998
Train RMSE:  5.990
------------------------------
Test Score:  0.875
Test RMSE:  45.358


From the optimization results above, we can see that achieving ~45 RMSE, ~0.87 $R^2$ is the best result we can get.

---
<a id='summary'></a>
## 3. Summary

From the plots and the RMSE, $R^2$ values, we can see that the prediction runs well when $0 \leq price \leq 300$ and as price prediction is less accurate when $300 < price$. One reason for higher prices listing to be inaccurate would be that any speciality of the house, like renovation, is only observable through listing picture and cannot be analyzed numerically or categorically. Further improvements can be made if we attempt to anaylze listing description using sentiment analysis etc.

In [17]:
# Save for analysis
pd.DataFrame(result).to_csv('data/model_results.csv')

In [18]:
observartion = pd.read_csv('data/model_results.csv', index_col=0)
obs_RMSE = observartion.sort_values(by='Test RMSE', ascending=False)
obs_R2 = observartion.sort_values(by='Test R2')

In [19]:
fig = make_subplots(rows=2, cols=1, subplot_titles=('Root Mean Square Error sorted by Test Data', 'R2 sorted by Test Data'))

fig.add_trace(go.Bar(name='Train', x=obs_RMSE['Method'], y=obs_RMSE['Train RMSE']), row=1, col=1)
fig.add_trace(go.Bar(name='Test', x=obs_RMSE['Method'], y=obs_RMSE['Test RMSE']), row=1, col=1)

fig.add_trace(go.Bar(name='Train', x=obs_R2['Method'], y=obs_R2['Train R2']), row=2, col=1)
fig.add_trace(go.Bar(name='Test', x=obs_R2['Method'], y=obs_R2['Test R2']), row=2, col=1)

fig.update_layout(height=800, width=800)
fig.show()